#TP3 : NLP Classification de texte

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import pandas as pd

# Define the path to your CSV file in Google Drive
file_path = '/content/drive/My Drive/Colab Notebooks/movie_review.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)


In [15]:
df.drop(columns=['fold_id','cv_tag','html_id'], inplace=True)
df


,sent_id,text,tag
0,0,films adapted from comic books have had plenty...,pos
1,1,"for starters , it was created by alan moore ( ...",pos
2,2,to say moore and campbell thoroughly researche...,pos
3,3,"the book ( or "" graphic novel , "" if you will ...",pos
4,4,"in other words , don't dismiss this film becau...",pos
...,...,...,...
64715,20,that lack of inspiration can be traced back to...,neg
64716,21,like too many of the skits on the current inca...,neg
64717,22,"after watching one of the "" roxbury "" skits on...",neg
64718,23,"bump unsuspecting women , and . . . that's all .",neg


In [16]:
import numpy as np
import re
import nltk
import spacy
import string

#Lower case the text
df["text"] = df["text"].str.lower()
df.head()

,sent_id,text,tag
0,0,films adapted from comic books have had plenty...,pos
1,1,"for starters , it was created by alan moore ( ...",pos
2,2,to say moore and campbell thoroughly researche...,pos
3,3,"the book ( or "" graphic novel , "" if you will ...",pos
4,4,"in other words , don't dismiss this film becau...",pos


In [18]:
#remove_punctuation

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["text"] = df["text"].apply(lambda text: remove_punctuation(text))
df.head()

,sent_id,text,tag
0,0,films adapted from comic books have had plenty...,pos
1,1,for starters it was created by alan moore an...,pos
2,2,to say moore and campbell thoroughly researche...,pos
3,3,the book or graphic novel if you will is ...,pos
4,4,in other words dont dismiss this film because...,pos


In [19]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

#remove_stopwords

STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["text"] = df["text"].apply(lambda text: remove_stopwords(text))
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,sent_id,text,tag
0,0,films adapted comic books plenty success wheth...,pos
1,1,starters created alan moore eddie campbell bro...,pos
2,2,say moore campbell thoroughly researched subje...,pos
3,3,book graphic novel 500 pages long includes nea...,pos
4,4,words dont dismiss film source,pos


In [20]:
#lemmatisation
import spacy

nlp = spacy.load("en_core_web_sm")

def lemmatize_words(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

df["text"] = df["text"].apply(lambda text: lemmatize_words(text))
df.head()

,sent_id,text,tag
0,0,film adapt comic book plenty success whether t...,pos
1,1,starter create alan moore eddie campbell bring...,pos
2,2,say moore campbell thoroughly research subject...,pos
3,3,book graphic novel 500 page long include nearl...,pos
4,4,word do not dismiss film source,pos


In [21]:
#tokenize the text :
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

df['text'] = df['text'].apply(lambda x: word_tokenize(x))
df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,sent_id,text,tag
0,0,"[film, adapt, comic, book, plenty, success, wh...",pos
1,1,"[starter, create, alan, moore, eddie, campbell...",pos
2,2,"[say, moore, campbell, thoroughly, research, s...",pos
3,3,"[book, graphic, novel, 500, page, long, includ...",pos
4,4,"[word, do, not, dismiss, film, source]",pos
...,...,...,...
64715,20,"[lack, inspiration, trace, back, insipid, char...",neg
64716,21,"[like, many, skit, current, incarnation, satur...",neg
64717,22,"[watch, one, roxbury, skit, snl, come, away, c...",neg
64718,23,"[bump, unsuspecting, woman, that, s]",neg


In [22]:
pip install gensim

In [23]:
import gensim
from gensim.models import Word2Vec
sentences = df['text'].tolist()
print(sentences)
# train word2vec on the two sentences
model = gensim.models.Word2Vec(sentences, min_count=0, sg=1, vector_size=100)  ##
model.save("word2vec_model2.model")
print(model)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Word2Vec<vocab=39317, vector_size=100, alpha=0.025>


In [24]:
vector = model.wv['film']
print(vector)

[ 0.05405021  0.36280492 -0.03162211  0.5761251   0.0118708  -0.45909873
  0.2854283   0.9311716  -0.52365506 -0.19877508  0.03907466 -0.39089817
 -0.09040835  0.28773913  0.24274735 -0.17683762  0.5725059   0.12552808
 -0.45471308 -0.791715    0.43264523 -0.03039729  0.77855176 -0.3031624
 -0.03406746 -0.08259045  0.03069331  0.07318199 -0.39131325  0.00845056
 -0.0702678   0.05626698  0.4952926  -0.7939831  -0.1186814   0.20383579
  0.22749163  0.01035954 -0.1838381  -0.20030987  0.1657567  -0.20606455
 -0.41751096  0.15509722  0.21627113  0.15531683  0.11095065  0.22802348
 -0.10405377  0.34351692  0.1059605  -0.0753082  -0.52998006 -0.3246381
 -0.15302901 -0.1588396   0.2727538   0.111151    0.0339998   0.27886093
  0.13621405  0.03848885  0.21877523 -0.16692373 -0.334478    0.5874755
  0.1911102   0.4294279  -0.5196123   0.3516013  -0.23079433  0.26657718
  0.04835952  0.05534375  0.07089064  0.17202887  0.21771264  0.1924452
 -0.26176137 -0.18667635 -0.15663388  0.22369248 -0.417

In [26]:
# Extraire les mots similaires à un mot central donné
similar_words = model.wv.most_similar('good', topn=20)

# 'similar_words' contient une liste de mots similaires
print(similar_words)


[('decent', 0.7922048568725586), ('mine', 0.7522284388542175), ('mediocre', 0.7517959475517273), ('notch', 0.7514412999153137), ('caliber', 0.7499686479568481), ('horrible', 0.7473784685134888), ('respectable', 0.7456406950950623), ('funniest', 0.7451596260070801), ('fantastic', 0.7447880506515503), ('accolade', 0.7404047846794128), ('flawless', 0.7390096783638), ('great', 0.7364740371704102), ('competent', 0.7364543676376343), ('awful', 0.7356019020080566), ('poor', 0.732541024684906), ('watchable', 0.732520580291748), ('nod', 0.732104480266571), ('worst', 0.7315914034843445), ('mainly', 0.7280247211456299), ('fine', 0.7258188724517822)]


#ML Model

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

X = df['text']
y = df['tag'].map({'pos': 1, 'neg': 0})

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


def average_word_vectors(words, model, vocabulary, num_features):
    feature_vector = np.zeros((num_features,), dtype="float64")
    nwords = 0.
    for word in words:
        if word in vocabulary:
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model.wv[word])
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
    return feature_vector

def word2vec_transform(data, model, vocabulary, num_features):
    vectors = [average_word_vectors(words, model, vocabulary, num_features) for words in data]
    return np.array(vectors)

# Vocabulary from the Word2Vec model
vocabulary = set(model.wv.index_to_key)

# Transforming text data to Word2Vec vectors
X_train_vec = word2vec_transform(X_train, model, vocabulary, 100)
X_test_vec = word2vec_transform(X_test, model, vocabulary, 100)

# Scaling the vectors
scaler = StandardScaler()
X_train_vec = scaler.fit_transform(X_train_vec)
X_test_vec = scaler.transform(X_test_vec)

# Training a classifier (e.g., Logistic Regression)
model = LogisticRegression()
model.fit(X_train_vec, y_train)

# Prediction
y_pred = model.predict(X_test_vec)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.5942521631644005
Classification Report:
               precision    recall  f1-score   support

           0       0.60      0.54      0.57      6371
           1       0.59      0.65      0.62      6573

    accuracy                           0.59     12944
   macro avg       0.59      0.59      0.59     12944
weighted avg       0.59      0.59      0.59     12944

